## What is an agent?

__Definition:__ The key behind agents is giving LLM's the possibility of using tools in their workflow. This is where langchain departs from the popular chatgpt implementation and we can start to get a glimpse of what it offers us as builders. 

The official definition of agents is the following:
- Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

### Agents and Tools

To use agents, we require three things:
- A base LLM,
- A tool that we will be interacting with,
- An agent to control the interaction.

In [2]:
#Let’s start by initializing our base LLM.
from langchain import OpenAI

llm = OpenAI(
    openai_api_key="",
    temperature=0,
    model_name="text-davinci-003"
)

When initializing tools, we either create a custom tool or load a prebuilt tool. In either case, the “tool” is a __utility chain__ given a tool __name__ and __description__.

In [ ]:
# For example, we could create a new calculator tool from the existing llm_math chain:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

In [6]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

We must follow this process when using custom tools. However, a prebuilt __llm_math__ tool does the same thing. So, we could do the same as above like so:

In [7]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [8]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

We now have the LLM and tools but no agent. To initialize a simple agent, we can do the following:

In [11]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

As the name suggests, we will use this agent to perform __'zero shot'__ tasks on the input. That means that we will not have several, interdependent interactions but only one. In other words, this agent will have no memory.

Now we are ready to initialize the agent! We will use verbose in True so we can see what is our agent's 'thinking' process.

__Important Note:__ When interacting with agents it is really important to set the max_iterations parameters because agents can get stuck in infinite loops that consume plenty of tokens. The default value is 15 to allow for many tools and complex reasoning but for most applications you should keep it much lower.

In [12]:
# With our agent initialized, we can begin using it. Let’s try a few prompts and see how the agent responds.
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

In [13]:
print("For verification:", (4.5*2.1)**2.2)

For verification: 139.94261298333066


In [14]:
# Let's try another
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...
 I need to figure out how many apples are in the boxes
Action: Calculator
Action Input: 8 * 2.5
Observation: Answer: 20.0
Thought: I need to add the apples Mary has to the apples in the boxes
Action: Calculator
Action Input: 4 + 20.0
Observation: Answer: 24.0
Thought: I now know the final answer
Final Answer: We have 24 apples.

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': 'We have 24 apples.'}

Looks great! But what if we decide to ask a non-math question? What if we ask an easy common knowledge question?

In [19]:
zero_shot_agent("what is the capital of Pakistan?")



> Entering new AgentExecutor chain...
 I need to look up the answer
Action: Look up
Action Input: Capital of Pakistan
Observation: Look up is not a valid tool, try one of [Calculator].
Thought: I need to find the answer online
Action: Search
Action Input: Capital of Pakistan
Observation: Search is not a valid tool, try one of [Calculator].
Thought: I need to find the answer in a database
Action: Database
Action Input: Capital of Pakistan
Observation: Database is not a valid tool, try one of [Calculator].
Thought:

> Finished chain.


{'input': 'what is the capital of Pakistan?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

We run into an error. The problem here is that the agent keeps trying to use a tool. Yet, our agent contains only one tool — the calculator.

Fortunately, we can fix this problem by giving our agent more tools! Let’s add a plain and simple LLM tool:

In [20]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

With this, we have a new general-purpose LLM tool. All we do is add it to the tools list and reinitialize the agent:

In [21]:
tools.append(llm_tool)

# reinitialize the agent
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

Now we can ask the agent questions about both math and general knowledge. Let’s try the following:

In [22]:
zero_shot_agent("what is the capital of Pakistan?")



> Entering new AgentExecutor chain...
 I need to find out the capital of Pakistan
Action: Language Model
Action Input: What is the capital of Pakistan?
Observation: 

The capital of Pakistan is Islamabad.
Thought: I now know the final answer
Final Answer: The capital of Pakistan is Islamabad.

> Finished chain.


{'input': 'what is the capital of Pakistan?',
 'output': 'The capital of Pakistan is Islamabad.'}

__Now we get the correct answer! We can ask the first question:__

In [23]:
zero_shot_agent("what is (4.5*2.1)^2.2?")



> Entering new AgentExecutor chain...
 I need to calculate this expression
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought: I now know the final answer
Final Answer: 139.94261298333066

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94261298333066'}

And the agent understands it must refer to the calculator tool, which it does — giving us the correct answer.

### Creating Database

We will use the agents to interact with a small sample database of stocks. We will now dive into the details because this is just a dummy tool we will build for illustrative purposes. Let's create it.

In [38]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [39]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)

In [40]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [41]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]
 

In [42]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [43]:
for obs in observations:
    insert_obs(obs)

In [46]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

## Agent types

In this section we will review several agents and see how they 'think' and what they can do.

### Agent type1: Zero Shot React

In [47]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=3
)

Let's see our newly created agent in action! We will ask it a question that involves a math operation over the stock prices.


In [48]:
result = agent_executor(
    "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " +
    "between the same stock prices in January the 4th?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: stocks
Thought: I should query the schema of the stocks table
Action: sql_db_schema
Action Input: stocks
Observation: 
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	price	date
1	ABC	200.0	2023-01-01
2	ABC	208.0	2023-01-02
3	ABC	232.0	2023-01-03
*/
Thought: I should query the ratio between the stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th
Action: sql_db_query
Action Input: SELECT (SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-03') / (SELECT price FROM stocks WHERE stock_ticker = 'XYZ' AND date = '2023-01-03') / (SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-04') / (SELECT price FROM stocks WHERE stock_ticker = 'XYZ' AND da

In [49]:
# let's see what the prompt is here:
print(agent_executor.agent.llm_chain.prompt.template)

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't k

Finally, let's pay attention to the __'agent_scratchpad'__. What is that? Well, that is where we will be appending every thought or action that the agent has already performed. In this way, at each point in time, the agent will know what it has found out and will be able to continue its thought process. In other words, after using a tool it adds its thoughts and observations to the scratchpad and picks up from there.

### Agent type2: Conversational React

The zero shot agent is really interesting but, as we said before, it has no memory. What if we want an assistant that remembers things we have talked about and can also reason about them and use tools? For that we have the conversational react agent.

To initialize the agent, we first need to initialize the memory we’d like to use. We will use the simple __ConversationBufferMemory__.

In [50]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [51]:
# We will use the math tool in this example and load it as below:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)

In [52]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [53]:
result = conversational_agent(
    "What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?"
    )



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 10,000, 8%, 5 years
Observation: Answer: 14693.280768000006
Thought: Do I need to use a tool? No
AI: The result of an investment of $10,000 growing at 8% annually for 5 years with compound interest is $14,693.28.

> Finished chain.


As we can see, the prompt is similar but it includes a great prelude of instructions that make it an effective assistant as well + a spot for including the chat history from the memory component:

In [54]:
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [55]:
# Let's see what happens if we try to answer the question that is related to the previous one:
result = conversational_agent(
    "If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?"
)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 15000, 8%, 5 years, compound interest
Observation: Answer: 22039.92115200001
Thought: Do I need to use a tool? No
AI: If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, we would have $22,039.92, which is $7,346.64 more than the previous scenario.

> Finished chain.


### Agent type3: Self Ask with Search

This is the first-choice agent to use when using LLM's to extract information with a search engine. The agent will ask follow-up questions and use the search functionality to get intermediate answers that help it get to a final answer.

In [56]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

# Creating a Google Search Tool using the SerpAPIWrapper (SerpApi is a real-time API to access Google search results) which requires an API key (you can generate it on their website)
search = SerpAPIWrapper(serpapi_api_key='111552fa19709a6e3919da9e05cea8718dcc53ab73bc8dcc503cd8e7c6b06bc5')
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)

In [57]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

As we can see, the prompt is basically a series of many examples to show the LLM how to ask follow up questions to a search tool until it can get to the final answe